In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


class FileProcess(object):
    def __init__(self, folder_address):
        self.folder_address = folder_address

    def find_dir(self):
        files_name = os.listdir(self.folder_address)
        filesPath = []
        for i in files_name:
            filesPath.append(self.folder_address + i)
        return filesPath


def DataPreprocessing():
    my_files = FileProcess('/Users/liyurun/Downloads/OneDrive_1_11-18-2020/提供データ/配達実績/')
    files_path = my_files.find_dir()
    df = pd.read_excel(files_path[10], usecols=[0, 1, 2, 5, 6, 7, 10, 11, 12, 13, 19, 20, 30])
    df_day = df[df['持出日付'] == 20200613]
    day_data = df_day[(df_day['配達乗務員\nコード'] != 0) & (df_day['緯度＿完了優先'] != 0) & (df_day['経度＿完了優先'] != 0)]
    day_data = day_data.reset_index(drop=True)
    day_data = day_data.sort_values(by=['配達乗務員\nコード', '持出時刻', '完了時刻'], axis=0)
    data_grouped = day_data.groupby(by='配達乗務員\nコード')
    driver_id = list(data_grouped.size().index)
    return driver_id, day_data


def DriverData(one_of_driver_id, day_data):  # input one of driver's id code
    return day_data[day_data['配達乗務員\nコード'] == one_of_driver_id]  # select the data of this driver


class LocationSearch(object):
    def __init__(self, driver_data, driver_id):
        self.Data = driver_data
        self.driver_id = driver_id
        self.start_time = driver_data['持出時刻'].values.tolist()
        self.number = driver_data['個数'].values.tolist()
        self.longitude = driver_data['経度＿完了優先'].values.tolist()
        self.latitude = driver_data['緯度＿完了優先'].values.tolist()
        self.end_time = driver_data['完了時刻'].values.tolist()

    def delivery(self):
        count = 0
        count_mark = []
        temp = self.end_time[0]
        for i in range(len(self.start_time)):
            if self.start_time[i] // 100 < temp:
                count_mark.append(count)
            else:
                count += 1
                temp = self.end_time[i]
                count_mark.append(count)

        loc_x = []
        loc_y = []
        for i in range(count + 1):
            loc_x.append([])
            loc_y.append([])
            loc_x[i].append(self.latitude[count_mark.index(i)])
            loc_y[i].append(self.longitude[count_mark.index(i)])

        for i in range(count + 1):
            if count_mark.count(i) > 1:
                for j in range(count_mark.index(i) + 1, count_mark.index(i) + count_mark.count(i)):
                    if self.end_time[j] != self.end_time[j - 1]:
                        loc_x[i].append(self.latitude[j])
                        loc_y[i].append(self.longitude[j])
        loc = []
        for i in range(count + 1):
            loc.append([])
            for j in range(len(loc_x[i])):
                loc[i].append([])
            for j in range(len(loc_x[i])):
                loc[i][j].append(loc_x[i][j])
                loc[i][j].append(loc_y[i][j])
        return loc, loc_x, loc_y

In [2]:
driver_id, day_data = DataPreprocessing()
driver_data = DriverData(driver_id[-1], day_data)
location = LocationSearch(driver_data, driver_id)
position = np.array(location.delivery()[0][0])

In [3]:
position

array([[ 35.3467755, 136.6335807],
       [ 35.3479707, 136.6304314],
       [ 35.3471914, 136.6341293],
       [ 35.350487 , 136.6297351],
       [ 35.3507923, 136.6305575],
       [ 35.3515581, 136.6336142],
       [ 35.3544407, 136.6327433],
       [ 35.3553003, 136.6363896],
       [ 35.3577956, 136.6363314],
       [ 35.3554103, 136.6267865],
       [ 35.3547903, 136.627796 ],
       [ 35.3537799, 136.6283168],
       [ 35.3615012, 136.6262586],
       [ 35.362901 , 136.624016 ]])